# MDAnalysis 2.6.1

import numpy as np

In [2]:
np.__version__

'1.22.4'

In [3]:
import pandas as pd

In [4]:
pd.__version__

'2.1.0'

In [5]:
import psutil

In [6]:
psutil.__version__

'5.9.0'

In [7]:
!echo $PATH

/home/ioana/anaconda3/envs/mdanalysis2.6.1/bin:/home/ioana/anaconda3/condabin:/home/ioana/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin


In [10]:
import MDAnalysis as mda

In [11]:
mda.__version__

'2.6.1'

# Load into MDAnalysis Universe

In [12]:
tprfile= "data/md_A4_dna.tpr"
trrfile= "data/md_A4_dna_xf.trr"
u=mda.Universe(tprfile,trrfile)

In [13]:
u

<Universe with 254 atoms>

In [14]:
#struc=u.select_atoms("all")
#struc.write("dna.pdb") # .tpr to .pdb file

# Trim the data to reduce analysis time

In [16]:
from CodeEntropy.IO import MDAUniverseHelper as MDAHelper
selection_string_pre_process= 'all'
start = 3
end = 40
step = 1

reduced_frame= MDAHelper.new_U_select_frame(u,start,end, step)

reduced_atom=MDAHelper.new_U_select_atom(reduced_frame, selection_string_pre_process)

In [17]:
reduced_atom

<Universe with 254 atoms>

# Parse the data into CodeEntropy Object

In [18]:
from CodeEntropy.ClassCollection import DataContainer as DC
dataContainer = DC.DataContainer (reduced_atom)

Number of atoms      : 254
Number of frames     : 37


# Calculations

In [19]:
from CodeEntropy.FunctionCollection import EntropyFunctions as EF
selection_string="all"
axis_list=["C5'","C4'","C3'"]
outfile = None
moutFile = None
nmdFile = None
csv_out = None
tScale= 1.0
fScale= 1.0
temper =300.0 #temperature = 300K
verbose =3
thread=4

## Molecule Level

In [20]:
wm_entropyFF, wm_entropyTT = EF.compute_entropy_whole_molecule_level(
    arg_hostDataContainer = dataContainer,
    arg_outFile = outfile,
    arg_selector = selection_string,
    arg_moutFile = moutFile,
    arg_nmdFile = nmdFile,
    arg_fScale = fScale,
    arg_tScale = tScale,
    arg_temper = temper,
    arg_verbose = verbose
)
print(f"wm_entropyFF={wm_entropyFF}")
print(f"wm_entropyTT={wm_entropyTT}")

------------------------------------------------------------
          Hierarchy level. --> Whole molecule <--           
------------------------------------------------------------
Total number of beads at the whole molecule level = 1
Assigning Translation and Rotation Axes @ whole molecule level-> Done
Updating Local coordinates-> Done
Updating Local forces-> Done
Updating Local torques-> Done
Weighting forces and torques-> Done
Updating the submatrices ... 
Finished updating submatrix FF
Finished updating submatrix TT
Done
Generating Quadrants-> Done
Diagonalizing-> Done
Changing the units of eigen values to SI units-> Done
Sorting spectrum in ascending order of frequencies-> Done
Entropy values:
FF Entropy (Whole mol level)             : 48.3374 J/mol/K
TT Entropy (Whole mol level)             : 43.2233 J/mol/K
wm_entropyFF=48.33739005621703
wm_entropyTT=43.22326663505889


## Residue Level

In [21]:
res_entropyFF, res_entropyTT = EF.compute_entropy_residue_level(
    arg_hostDataContainer = dataContainer,
    arg_outFile = outfile,
    arg_selector = selection_string,
    arg_moutFile = moutFile,
    arg_nmdFile = nmdFile,
    arg_fScale = fScale,
    arg_tScale = tScale,
    arg_temper = temper,
    arg_verbose = verbose,
    arg_axis_list=axis_list,
)
print(f"res_entropyFF={res_entropyFF}")
print(f"res_entropyTT={res_entropyTT}")

------------------------------------------------------------
             Hierarchy level. --> Residues <--              
------------------------------------------------------------
DA1
DA2
DA3
DA4
DT5
DT6
DT7
DT8
Total number of beads at the residue level = 8
Assigning Translation Axes @ residue level-> Done
Assigning Rotational Axes @ residue level->
    1     2     3     4     5 
    6     7     8 
Done
Updating Local forces-> Done
Updating Local coordinates-> Done
Updating Local torques-> Done
Weighting forces and torques-> Done
Updating the submatrices ... 
Finished updating submatrix FF
Finished updating submatrix TT
Done
Generating Quadrants-> Done
Diagonalizing-> Done
Changing the units of eigen values to SI units-> Done
Sorting spectrum in ascending order of frequencies-> Done
Entropy values:
FF Entropy (Residue level)               : 202.1649 J/mol/K
TT Entropy (Residue level)               : 311.2150 J/mol/K
res_entropyFF=202.1648873070285
res_entropyTT=311.21497333032397


## United Atom Level (Multi process - to speed up operation)

In [22]:
UA_entropyFF, UA_entropyTT, res_df = EF.compute_entropy_UA_level_multiprocess(
    arg_hostDataContainer = dataContainer,
    arg_outFile = outfile,
    arg_selector = selection_string,
    arg_moutFile = moutFile,
    arg_nmdFile = moutFile,
    arg_fScale = fScale,
    arg_tScale = tScale,
    arg_temper = temper,
    arg_verbose = verbose,
    arg_axis_list = axis_list,
    arg_thread=thread,
)
print(f"UA_entropyFF = {UA_entropyFF}")
print(f"UA_entropyTT = {UA_entropyTT}")
print("Per residue entropy:")
print(res_df)

Finished updating submatrix FF
Finished updating submatrix TT
A shape change has occured (54,54) -> (21, 21)



Finished updating submatrix FF
Finished updating submatrix TT
Finished updating submatrix FF
A shape change has occured (63,63) -> (19, 19)
Finished updating submatrix FF



Finished updating submatrix TT
Finished updating submatrix TT
A shape change has occured (63,63) -> (21, 21)


A shape change has occured (63,63) -> (19, 19)




Finished updating submatrix FF
Finished updating submatrix TT
A shape change has occured (51,51) -> (21, 21)



Finished updating submatrix FF
Finished updating submatrix TT
A shape change has occured (60,60) -> (19, 19)
Finished updating submatrix FF



Finished updating submatrix FF
Finished updating submatrix TT
A shape change has occured (60,60) -> (19, 19)



Finished updating submatrix TT
A shape change has occured (60,60) -> (21, 21)



  RESNAME  RESID  FF_ENTROPY(J/mol/K)  TT_ENTROPY(J/mol/K)
0      DA      1            44.510450        

In [24]:
print(UA_entropyFF)
print(UA_entropyTT)

342.2378738899839
214.6851895681265


In [25]:
result_entropyAEM = EF.compute_topographical_entropy_AEM(
    arg_hostDataContainer = dataContainer,
    arg_selector = selection_string,
    arg_outFile = outfile,
    arg_verbose = verbose
)

print(f"result_entropyAEM = {result_entropyAEM}")

------------------------------------------------------------
Topographical entropy of residue side chains 
computed using all the dihedrals with AEM method
------------------------------------------------------------
----------Working on resid : 1 (DA)----------
Found 18 exclusive dihedrals in residue DA1
Found 18 dihedrals which collectively acquire 7 unique conformers
Residue Topographical Entropy from AEM (DA 1) : 12.7386
----------Working on resid : 2 (DA)----------
Found 20 exclusive dihedrals in residue DA2
Found 20 dihedrals which collectively acquire 4 unique conformers
Residue Topographical Entropy from AEM (DA 2) : 7.6987
----------Working on resid : 3 (DA)----------
Found 20 exclusive dihedrals in residue DA3
Found 20 dihedrals which collectively acquire 2 unique conformers
Residue Topographical Entropy from AEM (DA 3) : 2.8480
----------Working on resid : 4 (DA)----------
Found 19 exclusive dihedrals in residue DA4
Found 19 dihedrals which collectively acquire 2 unique conf

In [26]:
total_entropy=wm_entropyFF+wm_entropyTT+res_entropyFF+res_entropyTT+ UA_entropyFF + UA_entropyTT + result_entropyAEM

In [27]:
print(total_entropy)

1196.1469354079857
